In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/cifar100/meta
/kaggle/input/cifar100/file.txt
/kaggle/input/cifar100/test
/kaggle/input/cifar100/train


# Transforming data

In [2]:
import torch 
import torchvision


transform_train = torchvision.transforms.Compose([
    torchvision.transforms.Resize(224),  # Resize to 256x256
    #torchvision.transforms.CenterCrop(64),  # Crop a central 224x224 region
    #torchvision.transforms.RandomResizedCrop(size=(64,64), scale=(0.8, 1.0)),  # Randomly resize and crop
    torchvision.transforms.RandomHorizontalFlip(p=0.5),  # Randomly flip horizontally
    torchvision.transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    torchvision.transforms.ToTensor(),  # Convert to PyTorch tensor
    torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize for ImageNet
])

transform_test=torchvision.transforms.Compose([
    torchvision.transforms.Resize(224),  
    torchvision.transforms.ToTensor(),  # Convert to PyTorch tensor
    torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Importing and Loading Data 

In [3]:
from torch.utils.data import Dataset, DataLoader
from torchvision.datasets import CIFAR100

root="./data"
train = CIFAR100(root=root, transform=transform_train,download=True,train=True)
test= CIFAR100(root=root, transform=transform_test,download=True,train=False)

Files already downloaded and verified
Files already downloaded and verified


# Initialize Dataloader

In [4]:
train_loader = DataLoader(train, batch_size=100, shuffle=True)
test_loader= DataLoader(test, batch_size=16, shuffle=True)

In [5]:
len(train_loader)

500

# Define Architecture

In [6]:
import torch.nn as nn

class VGG16(nn.Module):
    def __init__(self, num_classes=10):
        super(VGG16, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU())
        self.layer2 = nn.Sequential(
            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(), 
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        self.layer3 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU())
        self.layer4 = nn.Sequential(
            nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        self.layer5 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU())
        self.layer6 = nn.Sequential(
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU())
        self.layer7 = nn.Sequential(
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        self.layer8 = nn.Sequential(
            nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU())
        self.layer9 = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU())
        self.layer10 = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        self.layer11 = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU())
        self.layer12 = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU())
        self.layer13 = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))
            
        
        self.fc = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(7*7*512, 4096),
            nn.ReLU())
        
        self.fc1 = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(4096, 4096),
            nn.ReLU())
        self.fc2= nn.Sequential(
            nn.Linear(4096, num_classes))
        
    def forward(self, x):
        out = self.layer1(x)
        #print(out.shape)
        out = self.layer2(out)
        #print(out.shape)
        out = self.layer3(out)
        #print(out.shape)
        out = self.layer4(out)
        #print(out.shape)
        out = self.layer5(out)
        #print(out.shape)
        out = self.layer6(out)
        #print(out.shape)
        out = self.layer7(out)
        #print(out.shape)
        out = self.layer8(out)
        #print(out.shape)
        out = self.layer9(out)
        #print(out.shape)
        out = self.layer10(out)
        #print(out.shape)
        out = self.layer11(out)
        #print(out.shape)
        out = self.layer12(out)
        #print(out.shape)
        out = self.layer13(out)
        #print(out.shape)
    
        out=out.flatten(start_dim=1)
        #print(out.shape)
        out = self.fc(out)
        #print(out.shape)
        
        out = self.fc1(out)
        #print(out.shape)
        out = self.fc2(out)
        #print(out.shape)
        
        return out

# Hyperparameter tuning

In [7]:
device='cuda'
num_classes = 100
num_epochs = 2
learning_rate = 0.001
model = VGG16(num_classes)
#model=nn.DataParallel(model)
model=model.to(device)


# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum = 0.7)  


# Train the model
total_step = len(train_loader)

# Traning and Testing loop

In [8]:
from tqdm import tqdm
total_step = len(train_loader)
patience=2
patience_count=0
minimum_loss=float('inf')

for epoch in range(num_epochs):
    
    for  images, labels in tqdm(train_loader):  
        correct = 0
        total = 0
        # Move tensors to the configured device
        images = images.to(device)
        labels = labels.to(device)
        
        model.train()
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Loss {loss.item()}" )
    print('Accuracy of the network on the {} validation images: {} %'.format(5000, 100 * correct / total))
        
    # Validation
    with torch.no_grad():
        correct = 0
        total = 0
        eval_loss=0
        for images, labels in tqdm(test_loader):
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            
            eval_loss+=criterion(outputs, labels).item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            del images, labels, outputs
        print(f" Loss Minimum {minimum_loss} Loss_current {eval_loss}")
            
        if eval_loss<=minimum_loss:
            patience_count=0
            minimum_loss=eval_loss
        else:
            patience_count+=1
            print(f"Patience Counter {patience_count}")
        print('Accuracy of the network on the {} validation images: {} %'.format(5000, 100 * correct / total))
        
        if patience_count>=patience:
            print(f"Early stopping initialized")
            break


100%|██████████| 500/500 [09:11<00:00,  1.10s/it]


Loss 4.32022762298584
Accuracy of the network on the 5000 validation images: 4.0 %


100%|██████████| 625/625 [00:44<00:00, 13.96it/s]


 Loss Minimum inf Loss_current 2711.293501853943
Accuracy of the network on the 5000 validation images: 4.85 %


100%|██████████| 500/500 [09:13<00:00,  1.11s/it]


Loss 3.8115530014038086
Accuracy of the network on the 5000 validation images: 14.0 %


100%|██████████| 625/625 [00:44<00:00, 13.91it/s]

 Loss Minimum 2711.293501853943 Loss_current 2485.2630360126495
Accuracy of the network on the 5000 validation images: 9.39 %


# Passing Radnom image to find Cnn output

In [14]:
model = VGG16(num_classes).to(device)
a=torch.randn(1,3,224,224)
a=a.to(device)
#center_crop = torchvision.transforms.CenterCrop(64)

# Apply the center cropping transform to the input image
#a = center_crop(a)

x=model(a)
